# OpenStreetMapData Case Study

**Author : Chaitanya Madala **

**Date : May 15, 2016 **

## Map Area

[Ahemdabad, Gujarat, India](https://en.wikipedia.org/wiki/Ahmedabad)

[DataSet](https://mapzen.com/data/metro-extracts/metro/ahmedabad_india/) : This Dataset which is extracted from website openstreetmap contains information about the city Ahemdabad, India

## Data Auditing
- As part of data auditing plan lets find out what are the different types of tags present in our data set, but also how many, to get the feeling on how much of which data we can expect to have in the map.

- Below are required imports which will be used throught the project.

In [11]:
import xml.etree.cElementTree as ET
import pprint

In [22]:
def count_tags(filename): 
   
    '''This function is written to count no of 
       different tags present in the given dataset'''
    
    dict_tags = {}
    for event,element in ET.iterparse(filename):
        tag = element.tag
        if tag in dict_tags:
            dict_tags[tag] += 1
        else:
            dict_tags[tag] = 1
            
    return dict_tags

tags = count_tags('ahmedabad_india1.osm')
print(tags)

{'bounds': 1, 'tag': 98131, 'node': 546085, 'nd': 634041, 'way': 81271, 'member': 2291, 'relation': 511, 'osm': 1}


- Now lets find out how many different users contributed to this Ahemdabad openstreetmap dataset. 

In [25]:
def count_users(filename):
    
    users_set = set()
    for event,element in ET.iterparse(filename):
        tag = element.tag
        if tag == 'node' or tag == 'relation' or tag == 'way':
             users_set.add(element.attrib['user'])
        element.clear()        
    return users_set

users = count_users('ahmedabad_india1.osm')
print('Number of users contributed: ',len(users))

Number of users contributed:  354
